<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [ ]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

**Read the data**

In [ ]:
PATH_TO_DATA = Path('../input/flight-delays-fall-2018/')

In [ ]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')

In [ ]:
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

In [ ]:
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

In [ ]:
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

In [ ]:
train_df.head()

In [ ]:
#train_df['Month']

In [ ]:
train_df.info()

In [ ]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_feature_mask = train_df.dtypes==object
categorical_cols = train_df.columns[categorical_feature_mask].tolist()
# instantiate OneHotEncoder
#ohe = OneHotEncoder(categorical_features = categorical_feature_mask, sparse=False ) 
# categorical_features = boolean mask for categorical columns
# sparse = False output an array not sparse matrix
# apply OneHotEncoder on categorical feature columns
#X_ohe = ohe.fit_transform(train_df) # It returns an numpy array
categorical_feature_mask1 = test_df.dtypes==object
categorical_cols1 = test_df.columns[categorical_feature_mask1].tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

In [ ]:
 test_df

In [ ]:
# apply le on categorical feature columns
 test_df[categorical_cols1] =   test_df[categorical_cols1].apply(lambda col: le.fit_transform(col))
 test_df[categorical_cols1].head(10)

In [ ]:
 test_df

In [ ]:
# apply le on categorical feature columns
 train_df[categorical_cols] =  train_df[categorical_cols].apply(lambda col: le.fit_transform(col))
 train_df[categorical_cols].head(10)

In [ ]:
# import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
# instantiate OneHotEncoder
ohe = OneHotEncoder(categorical_features = categorical_feature_mask, sparse=False ) 
# categorical_features = boolean mask for categorical columns
# sparse = False output an array not sparse matrix
ohe1 = OneHotEncoder(categorical_features = categorical_feature_mask1, sparse=False ) 

In [ ]:
X_ohe = ohe.fit_transform(train_df) # It returns an numpy array

In [ ]:
X_ohe1 = ohe1.fit_transform(test_df)

In [ ]:
X_ohe1

In [ ]:
X_ohe

In [ ]:
X_train = X_ohe
#y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
y_train = train_df['dep_delayed_15min'].values
X_test = 

In [ ]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=42)

In [ ]:
ctb = CatBoostClassifier(random_seed=42, silent=False)

In [ ]:
%%time
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx);

In [ ]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

In [ ]:
roc_auc_score(y_valid, ctb_valid_pred)

In [ ]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx);

In [ ]:
ctb_test_pred = ctb.predict_proba(X_ohe1)[:, 1]

In [ ]:
roc_auc_score(y_valid, ctb_test_pred)